In [1]:
%pip install ipywidgets
%pip install ipympl

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.proj3d import proj_transform
from matplotlib import cbook, cm
from matplotlib.colors import LightSource
import matplotlib.tri as mtri
from matplotlib.patches import FancyArrowPatch, ArrowStyle
from time import sleep


from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

#%matplotlib widget
#%matplotlib inline

class Arrow3D(FancyArrowPatch):

    def __init__(self, x, y, z, dx, dy, dz, *args, **kwargs):
        super().__init__((0, 0), (0, 0), *args, **kwargs)
        self._xyz = (x, y, z)
        self._dxdydz = (dx, dy, dz)

    def draw(self, renderer):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        super().draw(renderer)
        
    def do_3d_projection(self, renderer=None):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))

        return np.min(zs) 

def _arrow3D(ax, x, y, z, dx, dy, dz, *args, **kwargs):
    '''Add an 3d arrow to an `Axes3D` instance.'''

    style = ArrowStyle('Fancy', head_length=2, head_width=2, tail_width=1)
    arrow = Arrow3D(x, y, z, dx, dy, dz, *args, arrowstyle=style,**kwargs)
    ax.add_artist(arrow)
    return arrow


def polarization(amp_ratio=1, phix=0, phiy=0, tmax=4):

    elev=20
    azim=-20
    roll=0
    
    tt = np.arange(0,  tmax*np.pi, 0.1)

    fig = plt.figure(figsize=(7,7))
    ax1 = fig.add_subplot(1,1,1, projection='3d')
    ax1.view_init(elev=elev, azim=azim)
    ax1.set_xlim(-5,4*np.pi)
    ax1.set_ylim(-1.1,1.1)
    ax1.set_zlim(-1.1,1.1)
    ax1.set_box_aspect(aspect=(8*np.pi,8,8))

    ax1.xaxis.set_ticklabels([])
    ax1.yaxis.set_ticklabels([])
    ax1.zaxis.set_ticklabels([])
    #ax1.set_axis_off()

# Make a mesh in the space of parameterisation variables u and v
    t = tt
    
    # This is the Mobius mapping, taking a u, v pair and returning an x, y, z
    # triple
    amp=np.sqrt(1+amp_ratio**2)
    x = 1/amp*np.sin(-t+phix*np.pi)
    y = amp_ratio/amp*np.sin(-t+phiy*np.pi)
    z = t
 
    # Plot the projection on the XY plane
    p1=ax1.plot(np.zeros_like(z)-5,  y, x, color='r', linestyle='solid',linewidth=1)
    p2=ax1.plot(np.zeros(50)-5, np.cos(np.linspace(0,2*np.pi,50)) , np.sin(np.linspace(0,2*np.pi,50)), color='k', 
             linestyle='dashed',linewidth=1)


    #ax2.plot(np.zeros_like(t),  y, x, color='r', linestyle='solid',linewidth=1)

    # Plot the 3D curve
    p3=ax1.plot(z, y, x, color='b',linewidth=1)
    #ax2.plot(t, y, np.zeros_like(x), label='3D curve', color='b',linewidth=1)
    #ax2.plot(t, np.zeros_like(y), x, label='3D curve', color='b',linewidth=1)

    
    #ax2.set_ylim(-1.1,1.1)
    #ax2.set_zlim(-1.1,1.1)
    
    #plt.show()

    p4=_arrow3D(ax1, t[-1],0,0,
           0,y[-1],x[-1],
           mutation_scale=6,
           ec ='blue',
           fc='red', alpha=1,lw=1)

    p5=_arrow3D(ax1, -5,0,0,
           0,y[-1],x[-1],
           mutation_scale=6,
           ec ='blue',
           fc='red', alpha=1,lw=1)

    for it, ix, iy in zip(t[0::2],x[0::2],y[0::2]):
        _arrow3D(ax1, it,0,0,
           0,iy,ix,
           mutation_scale=6,
           ec ='blue',
           fc='red', alpha=0.2,lw=1)

    plt.show()

 
interact(polarization, tmax=(0,4,0.1),amp_ratio=(0,1,0.1),phix=(-1,1,0.1),phiy=(-1,1,0.1));



interactive(children=(FloatSlider(value=1.0, description='amp_ratio', max=1.0), FloatSlider(value=0.0, descrip…